# PyMC3 report: Logistic regression spike slab prior

## PyMC3 introduction
Indeed discrete prior might also be not optimal for `PyMC3`, as pointed out [in this notebook](https://www.kaggle.com/derekpowll/bayesian-lr-w-cauchy-prior-in-pymc3).

Use `varbvs` analysis, $\pi_0 = 0.043, \mu = 0.777, \sigma = 0.844$.

Model
```
        xi = pm.Bernoulli('xi', pi0, shape = X.shape[1]) #inclusion probability for each variable
        alpha = pm.Normal('alpha', mu = mu_intercept, sd = sigma_intercept) # Intercept
        beta = pm.Normal('beta', mu = mu, sd = sigma , shape = X.shape[1]) #Prior for the non-zero coefficients
        p = pm.math.dot(X, xi * beta) #Deterministic function to map the stochastics to the output
        y_obs = pm.Bernoulli('y_obs', invlogit(p + alpha),  observed = y)  #Data likelihood
```
That is
```
        xi ∼ Bernoulli(p=0.043)
        alpha ~ Normal(mu=0.0, sigma=1.5)
        beta ~ Normal(mu=0.777, sigma=0.844)
        y_obs ~ Bernoulli(p=𝑓(𝑓(), 𝑓(𝑓(), 𝑓(𝑓(𝑓(𝑓(𝑎𝑟𝑟𝑎𝑦, 𝑓(xi, beta)), 𝑓(alpha)))))))
```

## PyMC3 performance
Performance is not stable when the causal variables themselves are correlated.

The number of samplers for all gene blocks is 2000.

### Block # 556_583
There are 28 genes in this block. Gene index #8, #14 and #20 are positive effect genes (effect 2.03, 0.84, 0.71). 20 consecutive genes sharing the same 2by2 table and then the same fisher test results (pvalue 1.296e-38). 3 out of 20 showed simulated positive effects. Inclusion probabilities for these 20 genes range from $0.1130$ to $0.1765$, very close to each other, cannot identify the real causal genes.

`susie` PIP for the 20 consecutive genes are the same, all around 0.05.

#### Fisher test results and effect size for simulation


    1st column: simulated gene name
    2nd column: number of cases with CNV
    3rd column: number of cases without CNV
    4th column: number of controls with CNV
    5th column: number of controls without CNV
    6th column: fisher test pvalue
    7th column: gene index in the block
    8th column: simulated effect size. 0 if not shown.

    gene_name   d_c	d_nc  nd_c  nd_nc	   p	gene_index	simulated_effect_size
    gene_567	132	6574	1	6705	1.296086e-38	10
    gene_582	132	6574	1	6705	1.296086e-38	25
    gene_581	132	6574	1	6705	1.296086e-38	24
    gene_580	132	6574	1	6705	1.296086e-38	23
    gene_577	132	6574	1	6705	1.296086e-38	20	0.71
    gene_579	132	6574	1	6705	1.296086e-38	22
    gene_576	132	6574	1	6705	1.296086e-38	19
    gene_575	132	6574	1	6705	1.296086e-38	18
    gene_574	132	6574	1	6705	1.296086e-38	17
    gene_566	132	6574	1	6705	1.296086e-38	9
    gene_573	132	6574	1	6705	1.296086e-38	16
    gene_571	132	6574	1	6705	1.296086e-38	14	0.84
    gene_570	132	6574	1	6705	1.296086e-38	13
    gene_569	132	6574	1	6705	1.296086e-38	12
    gene_563	132	6574	1	6705	1.296086e-38	6
    gene_564	132	6574	1	6705	1.296086e-38	7
    gene_565	132	6574	1	6705	1.296086e-38	8	 2.03
    gene_578	132	6574	1	6705	1.296086e-38	21
    gene_568	132	6574	1	6705	1.296086e-38	11
    gene_572	132	6574	1	6705	1.296086e-38	15
    gene_558	111	6595	1	6705	2.773089e-32	1
    gene_562	111	6595	1	6705	2.773089e-32	5
    gene_559	111	6595	1	6705	2.773089e-32	2
    gene_560	111	6595	1	6705	2.773089e-32	3
    gene_561	111	6595	1	6705	2.773089e-32	4
    gene_583	38	 6668	0	6706	6.903390e-12	26
    gene_584	21	 6685	0	6706	9.388358e-07	27
    gene_557	24	 6682	1	6705	1.520613e-06	0

#### PyMC3 results
    1st column: gene index in the block
    2nd column: inclusion probability
    3rd column: beta; mean(beta)
    4th column: beta given inclusion probability; sum(beta * inclusion probability) / sum(inclusion probability)

    gene_index	inclusion_probability	beta	beta_given_inclusion
    13	0.1765	0.912673	1.541970
    17	0.1680	0.899262	1.533234
    10	0.1565	0.894481	1.473024
    22	0.1450	0.899563	1.550085
    15	0.1400	0.881196	1.520979
    18	0.1390	0.877870	1.524457
    11	0.1365	0.872064	1.475609
    16	0.1355	0.876335	1.551596
    8	 0.1335	0.881722	1.562205
    20	0.1325	0.879677	1.423774
    9	 0.1325	0.879466	1.524186
    25	0.1320	0.896319	1.445784
    21	0.1315	0.870917	1.529550
    24	0.1315	0.879215	1.536151
    14	0.1315	0.889288	1.498121
    12	0.1300	0.882583	1.529814
    23	0.1280	0.862068	1.457598
    6	 0.1225	0.834628	1.497794
    19	0.1190	0.860485	1.423408
    7	0.1130	0.842922	1.405974
    5	0.0695	0.806391	1.038744
    3	0.0615	0.788237	1.092125
    2	0.0585	0.787436	0.997605
    26	0.0570	0.819494	1.083626
    1	0.0540	0.783500	0.949915
    4	0.0535	0.785115	0.938133
    27	0.0500	0.769003	0.961894
    0	0.0245	0.754623	0.167191

### Block # 666_677
There are 12 genes in the block. 3 out of 12 showed positive simulated effect. Gene index #4 and #5 share the same fisher test results and their inclusion probabilities are both around $0.5$. Gene index #9, #10, #11 also share the same fisher test results and their inclusion probabilities are all around $0.10$.
#### Fisher test results and effect size for simulation

    gene_name  d_c	d_nc   nd_c  nd_nc   p		gene_index	simulated_effect_size 
    gene_671	99	6607	9	6697	1.956916e-20	4 	 1.28
    gene_672	99	6607	9	6697	1.956916e-20	5 	 0.53
    gene_676	20	6686	0	6706	1.880480e-06	9
    gene_678	20	6686	0	6706	1.880480e-06	11	 0.98
    gene_677	20	6686	0	6706	1.880480e-06	10
    gene_673	23	6683	5	6701	9.016016e-04	6
    gene_674	23	6683	5	6701	9.016016e-04	7
    gene_675	23	6683	5	6701	9.016016e-04	8
    gene_667	12	6694	2	6704	1.289476e-02	0
    gene_668	17	6689	6	6700	3.453722e-02	1
    gene_670	17	6689	6	6700	3.453722e-02	3
    gene_669	17	6689	6	6700	3.453722e-02	2

#### PyMC3 results
    gene_index	inclusion_probability	beta	beta_given_inclusion
    4	0.5460	1.477352	2.060144
    5	0.5185	1.432267	2.043380
    10	0.1325	0.848880	1.329551
    9	0.1270	0.854411	1.342899
    11	0.1080	0.823739	1.277766
    8	0.0300	0.750469	0.429703
    0	0.0240	0.744307	0.225855
    7	0.0230	0.770120	0.375470
    1	0.0195	0.749606	0.191625
    6	0.0185	0.788487	0.367600
    3	0.0150	0.744767	0.102463
    2	0.0110	0.773154	0.384195

### Block # 1018_1031
There are 14 genes in the block. Gene index #2 and #13 showed simulated positive effects. Fisher test Pvalue for gene #2 is uniquely smallest, and it inclusion probability (0.3040) is the largest.
#### Fisher test results and effect size for simulation
    gene_name	d_c  d_nc	nd_c  nd_nc	p	gene_index	simulated_effect_size 
    gene_1021	14   6692	3	6703	0.012672	2		0.60
    gene_1023	7	6699	2	6704	0.179541	4
    gene_1024	7	6699	2	6704	0.179541	5
    gene_1022	7	6699	2	6704	0.179541	3
    gene_1025	7	6699	2	6704	0.179541	6
    gene_1027	7	6699	2	6704	0.179541	8
    gene_1030	7	6699	2	6704	0.179541	11
    gene_1031	7	6699	2	6704	0.179541	12
    gene_1032	7	6699	2	6704	0.179541	13		1.07
    gene_1029	7	6699	2	6704	0.179541	10
    gene_1026	7	6699	2	6704	0.179541	7
    gene_1028	7	6699	2	6704	0.179541	9
    gene_1020	9	6697	4	6702	0.266611	1
    gene_1019	9	6697	4	6702	0.266611	0

#### PyMC3 results
    gene_index	inclusion_probability	beta	beta_given_inclusion
    2	0.3040	0.926048	1.296915
    5	0.0650	0.767650	0.801550
    8	0.0640	0.777537	0.820872
    9	0.0635	0.773069	0.866680
    10	0.0585	0.791200	0.832734
    3	0.0580	0.775015	0.843443
    12	0.0580	0.796682	0.831028
    13	0.0580	0.780965	0.919135
    4	0.0505	0.768463	0.766376
    6	0.0495	0.774164	0.727036
    11	0.0485	0.796748	0.973925
    7	0.0475	0.780096	0.834351
    0	0.0385	0.804297	0.855654
    1	0.0365	0.755608	0.538007

### block 23_36
There are 14 genes in the block. 4 out of 14 have positive effects (0.98, 1.75, 0.01, 0.17). Gene index #1 to #5 share the same fisher test results (1.910e-107). We expect their inclusion probabilities are close. But only #4 and #5 showed larger inclusion probabilities, 0.635 and 0.310. In contrast, inclusion probability for #2 fell to 0.

When performing 2nd time, gene #1, 2, 4, 5 showed inclusion probabilities greater than 0.20, #3 around 0.09. When consecutive genes share the same CNV overlap situations, they showed randomness with respect to the rank of inclusion probability.
#### Fisher test results and effect size for simulation
    gene_name  d_c	d_nc	nd_c  nd_nc	p			gene_index	simulated_effect_size
    gene_25	603	6103	76	6630	1.910490e-107	1
    gene_28	603	6103	76	6630	1.910490e-107	4
    gene_27	603	6103	76	6630	1.910490e-107	3
    gene_26	603	6103	76	6630	1.910490e-107	2			1.75
    gene_29	603	6103	76	6630	1.910490e-107	5			0.01
    gene_24	528	6178	61	6645	1.710063e-97	 0			0.98
    gene_30	234	6472	41	6665	1.291823e-34	 6
    gene_31	162	6544	26	6680	1.641138e-25	 7
    gene_32	20	 6686	3	 6703	4.832617e-04	 8
    gene_33	20	 6686	3	 6703	4.832617e-04	 9
    gene_34	20	 6686	3	 6703	4.832617e-04	 10
    gene_37	12	 6694	1	 6705	3.403787e-03	 13
    gene_36	12	 6694	1	 6705	3.403787e-03	 12
    gene_35	12	 6694	1	 6705	3.403787e-03	 11			0.17

#### PyMC3 results
    gene_index	inclusion_probability	beta	beta_given_inclusion
    4	0.635	1.632173	2.081691
    5	0.310	1.125279	1.926215
    3	0.080	0.914193	1.923941
    1	0.060	0.830937	1.141560
    6	0.060	0.648318	-0.450090
    12	0.050	0.841260	0.538263
    0	0.030	0.810724	0.493963
    8	0.025	0.714397	0.013209
    11	0.025	0.820051	0.485907
    13	0.020	0.805907	0.752812
    7	0.010	0.758125	0.016305
    10	0.010	0.823287	0.027814
    9	0.005	0.737005	0.867159
    2	0.000	0.865975	NaN

#### PyMC3 results (perform 2nd time)
    gene_index	inclusion_probability	beta	beta_given_inclusion
    4	0.3590	1.169565	1.899823
    5	0.2765	1.086049	1.846355
    2	0.2050	0.992402	1.785732
    1	0.2025	1.013238	1.850048
    0	0.1350	0.763805	0.609544
    3	0.0900	0.860466	1.652932
    6	0.0585	0.704812	-0.448460
    11	0.0410	0.792157	0.704093
    12	0.0360	0.784724	0.668371
    13	0.0355	0.792590	0.685245
    10	0.0265	0.755249	0.193445
    9	0.0195	0.760171	0.174600
    8	0.0185	0.733963	0.070547
    7	0.0090	0.762858	-0.069858